In [111]:
%load_ext autoreload
%autoreload 2

import numpy as np
import random
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import os
from sklearn.feature_selection import RFECV
import pandas as pd
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    KFold,
)
from sklearn.metrics import f1_score
import numpy as np
from numpy.lib.function_base import average
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    recall_score,
    f1_score,
    precision_score,
    confusion_matrix,
    make_scorer,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    KFold,
)
from sklearn.feature_selection import SelectKBest, SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import importlib
import os
from sklearn.feature_selection import (
    SelectKBest,
    RFE,
    mutual_info_regression,
    f_regression,
    mutual_info_classif,
)
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.inspection import permutation_importance
import xgboost as xgb
import random
import pickle
import openpyxl
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [127]:
# Setting path to the initial folder
os.chdir("C:\\Users\\Marieke\\GitHub\\Typhoon_IBF_Rice_Damage_Model")
cdir = os.getcwd()

import importlib

from IBF_typhoon_model.models.classification.xgb_binary import xgb_binary_features
from IBF_typhoon_model.models.classification.rf_binary import rf_binary_features, rf_binary_performance
from IBF_typhoon_model.models.classification.xgb_binary import xgb_binary_features, xgb_binary_performance

from IBF_typhoon_model.models.utility_functions.splitting_train_test import splitting_train_test
from IBF_typhoon_model.models.utility_functions.determine_class import determine_class

<IPython.core.display.Javascript object>

# Notebook to obtain the model performance

## Loading the Dataset

In [75]:
# Input data
name = "IBF_typhoon_model\\data\\restricted_data\\combined_input_data\\input_data_new.xlsx"
path = os.path.join(cdir, name)
df = pd.read_excel(path, engine="openpyxl")

# Typhoon overview
file_name = "IBF_typhoon_model\\data\\restricted_data\\data_overview.xlsx"
path = os.path.join(cdir, file_name)
df_typh_overview = pd.read_excel(path, sheet_name="typhoon_overview", engine="openpyxl")

In [76]:
# Selecting the features to be used
features = [
    "rice_area",
    "mean_slope",
    "mean_elevation_m",
    "ruggedness_stdev",
    "mean_ruggedness",
    "slope_stdev",
    "area_km2",
    "poverty_perc",
    "with_coast",
    "coast_length",
    "perimeter",
    "glat",
    "glon",
    "coast_peri_ratio",
    "rainfall_sum",
    "rainfall_max",
    "dis_track_min",
    "vmax_sust",
]

# Binary Classification

In [ ]:
# Setting the general input variables
threshold = 0.3
df["class_value_binary"] = [1 if df["perc_loss"][i] > threshold else 0 for i in range(len(df))]

### Feature selection

In [ ]:
X = df[features]
y = df['class_value_binary']

In [77]:
# Setting the random forest search grid
rf_search_space = [{
        "estimator__n_estimators": [50, 100, 500],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10],
        "estimator__min_samples_leaf": [1, 3, 5],
    }]

selected_features_rf = rf_binary_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    cv_splits=5,
    class_weight='balanced',
    min_features_to_select=7,
    GS_score='f1',
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features RF: {selected_features_rf}")

In [ ]:
# Setting the XGBoost search grid
xgb_search_space = [{
        "estimator__learning_rate": [0.1, 0.5, 1]
        "estimator__gamma": [0.1, 0.5, 2],
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.01, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7, 1],
    }]

selected_features_xgb = xgb_binary_features(
    X=X, 
    y=y, 
    features=features, 
    search_space=xgb_search_space, 
    objective="binary:hinge", 
    cv_splits=5, 
    min_features_to_select=7, 
    GS_score='f1', 
    GS_n_iter=30
)

print(f"Selected features XGBoost: {selected_features_xgb}")

### Obtain performance estimate

In [83]:
# Setting the train and the test sets
df_train_list, df_test_list = splitting_train_test(df)

In [84]:
# Setting the selected features for RF --> if needed
selected_features_rf = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [115]:
# Setting the random forest search grid
rf_search_space = [{
        "rf__n_estimators": [50, 100, 500],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

# Obtaining the performance estimate
df_predicted_rf = rf_binary_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf,
    search_space=rf_search_space,
    stratK=True,
    cv_splits=5,
    class_weight='balanced',
    GS_score='f1',
    GS_randomized=True,
    GS_n_iter=10,
)

Running for 0 out of a total of 5
Train score: 0.8469775474956822
Test score: 0.703125
Running for 1 out of a total of 5
Train score: 0.8534752661239826
Test score: 0.6037735849056604
Running for 2 out of a total of 5
Train score: 0.8504941026458399
Test score: 0.4770642201834863
Running for 3 out of a total of 5
Train score: 0.8653421633554085
Test score: 0.608058608058608
Running for 4 out of a total of 5
Train score: 0.8774152676591701
Test score: 0.3225806451612903


<IPython.core.display.Javascript object>

In [93]:
# Setting the selected features for XGB
selected_features_xgb = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

<IPython.core.display.Javascript object>

In [124]:
# Setting the XGBoost search grid
xgb_search_space = [{
        "xgb__learning_rate": [0.1, 0.5, 1],
        "xgb__gamma": [0.1, 0.5, 2],
        "xgb__max_depth": [6, 8],
        "xgb__reg_lambda": [0.001, 0.01, 0.1, 1],
        "xgb__n_estimators": [100, 200],
        "xgb__colsample_bytree": [0.5, 0.7, 1],
    }]

df_predicted_xgb = xgb_binary_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_xgb,
    search_space=xgb_search_space,
    stratK=True,
    cv_splits=5,
    objective="binary:hinge",
    GS_score='f1',
    GS_randomized=True,
    GS_n_iter=10,
)

Running for 0 out of a total of 5
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.5, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.001
[CV 1/5; 1/10] END xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.5, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.001;, score=(train=0.942, test=0.574) total time=   0.3s
[CV 2/5; 1/10] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.5, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.001
[CV 2/5; 1/10] END xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.5, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.001;, score=(train=0.974, test=0.510) total time=   0.4s
[CV 3/5; 1/10] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.5, xgb__max_depth=6, xgb__n_estimators=100, xgb__reg_lambda=0.001
[CV 3/5; 1/10] END xgb__colsample_bytree=0.7, xg

<IPython.core.display.Javascript object>

In [133]:
a = list(df_predicted_xgb['actual'])
b = list(df_predicted_xgb['predicted'])
f1_score(a, b)

0.4953703703703704

<IPython.core.display.Javascript object>

# Multiclass Classification

In [132]:
# Setting class value
classes = {"0": [0, 0.3], "1": [0.3, 0.8], "2": [0.8, 1.1]}
df['class_value_multi'] = df['perc_loss'].apply(lambda x: determine_class(x, classes=classes))

<IPython.core.display.Javascript object>

## Feature Selection

In [ ]:
X = df[features]
y = df['class_value_multi']

In [ ]:
# Setting the random forest search grid
rf_search_space = [{
        "estimator__n_estimators": [50, 100, 500],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10],
        "estimator__min_samples_leaf": [1, 3, 5],
    }]

selected_features_rf = rf_multi_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    cv_splits=5,
    class_weight='balanced',
    min_features_to_select=7,
    GS_score='f1_macro',
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features RF: {selected_features_rf}")

In [ ]:
# Setting the XGBoost search grid
xgb_search_space = [{
        "estimator__learning_rate": [0.1, 0.5, 1]
        "estimator__gamma": [0.1, 0.5, 2],
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.01, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7, 1],
    }]

selected_features_xgb = xgb_multi_features(
    X=X,
    y=y,
    features=selected_features_xgb,
    num_class=len(classes),
    search_space=xgb_search_space,
    objective="multi:softmax",
    cv_splits=5,
    min_features_to_select=7,
    GS_score='f1_macro',
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features XGBoost: {selected_features_xgb}")

## Obtain performance estimate

In [135]:
# Setting the train and the test sets
df_train_list, df_test_list = splitting_train_test(df)

<IPython.core.display.Javascript object>

In [ ]:
# Setting the selected features for RF --> if needed
selected_features_rf = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
# Setting the random forest search grid
rf_search_space = [{
        "rf__n_estimators": [50, 100, 500],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

df_predicted_rf = rf_multi_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf,
    search_space=rf_search_space,
    stratK=True,
    cv_splits=5,
    class_weight='balanced',
    GS_score='f1_macro',
    GS_randomized=True,
    GS_n_iter=10,
)

In [ ]:
# Setting the selected features for XGB
selected_features_xgb = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
# Setting the XGBoost search grid
xgb_search_space = [{
        "xgb__learning_rate": [0.1, 0.5, 1],
        "xgb__gamma": [0.1, 0.5, 2],
        "xgb__max_depth": [6, 8],
        "xgb__reg_lambda": [0.001, 0.01, 0.1, 1],
        "xgb__n_estimators": [100, 200],
        "xgb__colsample_bytree": [0.5, 0.7, 1],
    }]

df_predicted_xgb = xgb_multi_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    num_class=len(classes),
    features=selected_features_xgb,
    search_space=xgb_search_space,
    stratK=True,
    cv_splits=5,
    objective="multi:softmax",
    GS_score='f1_macro',
    GS_randomized=True,
    GS_n_iter=10,
)

# Regression

## Feature selections

In [ ]:
X = df[features]
y = df['perc_loss']

In [ ]:
#%% Setting input varialbes
rf_search_space = [{
        "estimator__n_estimators": [50, 100, 500],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10],
        "estimator__min_samples_leaf": [1, 3, 5],
    }]

selected_features_rf = rf_regression_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    min_features_to_select=7,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features RF: {selected_features_rf}")

In [ ]:
xgb_search_space = [{
        "estimator__learning_rate": [0.1, 0.5, 1],
        "estimator__gamma": [0.1, 0.5, 2],
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.01, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7, 1],
    }]

selected_features_xgb = xgb_regression_features(
    X=X,
    y=y,
    features=features,
    search_space=xgb_search_space,
    min_features_to_select=7,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    objective='"reg:squarederror"',
    GS_randomized=True,
    GS_n_iter=10,
)

print(f"Selected features RF: {selected_features_xgb}")

## Obtaining performance Estimate

In [ ]:
# Setting the train and the test sets
df_train_list, df_test_list = splitting_train_test(df)

In [ ]:
# Setting the selected features for rf
selected_features_rf = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
#%% Setting input varialbes
rf_search_space = [{
        "rf__n_estimators": [50, 100, 500],
        "rf__max_depth": [20, None],
        "rf__min_samples_split": [2, 8, 10],
        "rf__min_samples_leaf": [1, 3, 5],
    }]

df_predicted_rf = rf_regression_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=selected_features_rf,
    search_space=rf_search_space,
    cv_splits=5,
    GS_score="neg_root_mean_squared_error",
    GS_randomized=True,
    GS_n_iter=10,
)

In [ ]:
# Setting the selected features for XGB
selected_features_xgb = [
    'rice_area', 
    'mean_slope', 
    'mean_elevation_m', 
    'ruggedness_stdev', 
    'mean_ruggedness', 
    'slope_stdev', 
    'area_km2', 
    'poverty_perc', 
    'with_coast', 
    'coast_length', 
    'perimeter', 
    'glat', 
    'glon', 
    'coast_peri_ratio', 
    'rainfall_sum', 
    'rainfall_max', 
    'dis_track_min', 
    'vmax_sust'
]

In [ ]:
xgb_search_space = [{
        "xgb__learning_rate": [0.1, 0.5, 1],
        "xgb__gamma": [0.1, 0.5, 2],
        "xgb__max_depth": [6, 8],
        "xgb__reg_lambda": [0.001, 0.01, 0.1, 1],
        "xgb__n_estimators": [100, 200],
        "xgb__colsample_bytree": [0.5, 0.7, 1],
    }]

df_predicted_xgb = xgb_regression_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    features=features,
    search_space=xgb_search_space,
    cv_splits=5,
    objective="reg:squarederror",
    GS_score="neg_root_mean_squared_error",
    GS_randomized=True,
    GS_n_iter=10
)